In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library


print('Libraries imported.')

/bin/bash: conda: command not found
Libraries imported.


In [3]:
#Reading the data from the internet
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
webpage = requests.get(url)

#**Question 1**

In [5]:
df_data_1 = pd.read_html(webpage.content, header=0)[0]
df_data_1.head()


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [10]:
#Remove not assigned data
df_data_1=df_data_1[df_data_1['Borough']!='Not assigned']
df_data_1.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [11]:
#Replace not assigned neighborhood with Borough label
df_data_2= df_data_1.groupby(['Postal Code', 'Borough'])['Neighbourhood'].apply(', '.join)
df_data_2.head()

Postal Code  Borough    
M1B          Scarborough                            Malvern, Rouge
M1C          Scarborough    Rouge Hill, Port Union, Highland Creek
M1E          Scarborough         Guildwood, Morningside, West Hill
M1G          Scarborough                                    Woburn
M1H          Scarborough                                 Cedarbrae
Name: Neighbourhood, dtype: object

In [12]:
df_data_2.shape

(103,)

#**Question 2**

In [15]:
geosdata = "https://cocl.us/Geospatial_data"
geodata = pd.read_csv(geospatial_url)
geodata.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [16]:
geodata.shape

(103, 3)

In [18]:
#Merge data on Postal Code
df_data_3 = pd.merge(df_data_2, geodata, on = 'Postal Code')
df_data_3.head()

,Postal Code,Neighbourhood,Latitude,Longitude
0,M1B,"Malvern, Rouge",43.806686,-79.194353
1,M1C,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Woburn,43.770992,-79.216917
4,M1H,Cedarbrae,43.773136,-79.239476




#**Question 3**




In [19]:
#Toronto corordinates
address = 'Toronto, Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The coordinates of Toronto are {}, {}.'.format(latitude, longitude))

/usr/local/lib/python3.6/dist-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The coordinates of Toronto are 43.6534817, -79.3839347.


In [29]:
# create map
map = folium.Map(location = [latitude,longitude], zoom_start=11)

# add markers to map
for lat, lng, stat in zip(df_data_3['Latitude'], df_data_3['Longitude'], df_data_3['Neighbourhood']):
    label = '{}'.format(stat)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map)  
map